Work in progress...

# Continual learning of sequence patterns with online learning algorithms

In this notebook, we show how to perform the continual training of a RNN model on a sequence generation task. We also provide the code for the hyperparameter optimization for those models.

## Data set

## Models

## Hyperparameter optimization on one model

## Evaluation of the obtained hyperparameters